In [2]:
# need python 3.12+ for typing
!python -V

Python 3.12.6


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from src.sokoban_solver import (
    SokobanEdge,
    SokobanNode,
    Solver,
    BFSQueue,
    PriorityQueue,
)
from src.heuristics import (
    f_priority_length,
    f_priority_l1_naive,
    f_priority_l1_symmetric_naive,
    f_priority_l1_matching,
    f_priority_dijkstra,
    f_priority_dijkstra_stuck_basic,
    f_priority_dlsb,
)
from boards.utilities import load_json_to_dict

## Demo

In [8]:
# loading a puzzle
json_dict = load_json_to_dict('boards/board_v=1&seed=1565730199&level=5.json')
node = SokobanNode(json_dict['airs'], json_dict['boxes'], json_dict['goals'], json_dict['player'])
print(node)

█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █         ● █ █ █
█ █ █ █ █ · █ █ █   █ █ █
█ █ █ █ █     █ █ □ █ █ █
█ █               □   █ █
█ █   █ █   █   █ █   █ █
█ █   █ █   █         █ █
█ █   █ █     ■ □ █ █ █ █
█ █           ·   █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █


In [9]:
# solving it: status code 200 means success!
solver = Solver(PriorityQueue(f_priority_dlsb), node)
solver.solve()

200

In [10]:
# here's the solution
print(solver.solution)

█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █         ● █ █ █
█ █ █ █ █ · █ █ █   █ █ █
█ █ █ █ █     █ █ □ █ █ █
█ █               □   █ █
█ █   █ █   █   █ █   █ █
█ █   █ █   █         █ █
█ █   █ █     ■ □ █ █ █ █
█ █           ·   █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █

to

█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █         ■ █ █ █
█ █ █ █ █ ■ █ █ █   █ █ █
█ █ █ █ █     █ █   █ █ █
█ █                   █ █
█ █   █ █   █   █ █   █ █
█ █   █ █   █ ○       █ █
█ █   █ █     ■   █ █ █ █
█ █           ■   █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █ █

via

Move (-1, 0) from (8, 4) to (7, 4)
Move (-1, 0) from (7, 4) to (6, 4)
Move (-1, 0) from (6, 4) to (5, 4)
Move (-1, 0) from (5, 4) to (4, 4)
Move (0, 1) from (7, 4) to (7, 5)
Move (0, 1) from (7, 5) to (7, 6)
Move (-1, 0) from (4, 4) to (3, 4)
Move (1, 0) from (1, 4) to (2, 4)
Move (0, 1) from (3, 3) to (3, 4)
Move (0, 1) from (3, 4) to (3, 5)
Move (0, 1) from (6,

## To do

**General**
- Better solution printing (Path objects)
    - Printing moves
    - Animating? .visualize() function?

**Generator**
- Figure out how the papers are doing things
- Implement something

**Solver**
- Implement more heuristics
    - Box pushing heuristic:
        - Upgrade of Dijkstra
        - Solving subpuzzles with just one box
        - Chained heuristics: solve subpuzzles with Dijkstra
        - **Also, this might be *really* slow: we have to consider each box/goal matchup to then do the linprog to find the smallest number of pushes. So that's n^2 subpuzzles to solve! It's probably best to just come up with pruning conditions based on how a box can be pushed (ex. if it's stuck in a dead-end hallway/component).**
        - Stuck box handling
            - Start with a stuck boxes pruning heuristic
            - Consider stuck-on-goal boxes to be equivalent to walls
            - Implement things reasonably flexibly: so if the notion of stuck (ex. basic vs "conditionally stuck but actually stuck") changes, it can still work
        - Include timeouts
            - How to handle these?
            - Fall back to Dijkstra?
            - Tuple priority with the first entry indicating timeout?
            - Option to repeat with longer timeouts on subsequent runs?
            - Tagging queue entries with solver objects so I can easily pick up heuristic evals once I exhaust more clearly viable paths?
            - **A general framework that manages priority calculation priorities (for multiple possible heuristics)? Maybe a secondary queue for that?**
    - Brainstorm more ideas
        - I should do more puzzles by hand to figure out next gen heuristics
- Ponder misc ideas
    - What would it look like to work neural nets into the problem? As the sole heuristic? For blending heuristics? For a CNN? What's the right learning framework? Reinforcement?
    - Splitting into sub-puzzles?